## Login Hugging Face

In [1]:
from huggingface_hub import notebook_login
notebook_login()

##  Load OPUS Books dataset

In [2]:
from datasets import load_dataset


books = load_dataset("opus_books", "en-fr")
books = books["train"].train_test_split(test_size=0.2)
books["train"][0]

{'id': '92904',
 'translation': {'en': 'Among other fowl I noted some sheathbills from the wading-bird family, the size of pigeons, white in color, the beak short and conical, the eyes framed by red circles.',
  'fr': "Parmi les oiseaux, je remarquai des chionis, de la famille des échassiers, gros comme des pigeons, blancs de couleur, le bec court et conique, l'oeil encadré d'un cercle rouge."}}

## Prepocessing

In [3]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

In [4]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [5]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

### If Using Pytorch

In [6]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### If Using Tensorflow

In [ ]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")

## Evaluate

In [7]:
import evaluate
metric = evaluate.load("sacrebleu")

In [8]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

## Train

### If using Pytorch

In [9]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [10]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [11]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/12710 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 2.091, 'learning_rate': 1.9213217938631003e-05, 'epoch': 0.08}
{'loss': 2.0135, 'learning_rate': 1.8426435877261997e-05, 'epoch': 0.16}
{'loss': 1.966, 'learning_rate': 1.7639653815893e-05, 'epoch': 0.24}
{'loss': 1.9308, 'learning_rate': 1.6852871754524e-05, 'epoch': 0.31}
{'loss': 1.9399, 'learning_rate': 1.6067663257277734e-05, 'epoch': 0.39}
{'loss': 1.9099, 'learning_rate': 1.5280881195908735e-05, 'epoch': 0.47}
{'loss': 1.8945, 'learning_rate': 1.4494099134539735e-05, 'epoch': 0.55}
{'loss': 1.9002, 'learning_rate': 1.3707317073170734e-05, 'epoch': 0.63}
{'loss': 1.8624, 'learning_rate': 1.292210857592447e-05, 'epoch': 0.71}
{'loss': 1.8742, 'learning_rate': 1.213532651455547e-05, 'epoch': 0.79}
{'loss': 1.8632, 'learning_rate': 1.1348544453186468e-05, 'epoch': 0.87}
{'loss': 1.8812, 'learning_rate': 1.0561762391817467e-05, 'epoch': 0.94}


/home/odhi/anaconda3/envs/sentiment-analysis-transformers-pytorch/lib/python3.11/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1589 [00:00<?, ?it/s]

{'eval_loss': 1.6232786178588867, 'eval_bleu': 5.5232, 'eval_gen_len': 17.5864, 'eval_runtime': 331.4869, 'eval_samples_per_second': 76.676, 'eval_steps_per_second': 4.794, 'epoch': 1.0}
{'loss': 1.8593, 'learning_rate': 9.778127458693943e-06, 'epoch': 1.02}
{'loss': 1.8607, 'learning_rate': 8.991345397324942e-06, 'epoch': 1.1}
{'loss': 1.8335, 'learning_rate': 8.20456333595594e-06, 'epoch': 1.18}
{'loss': 1.8441, 'learning_rate': 7.4177812745869405e-06, 'epoch': 1.26}
{'loss': 1.8286, 'learning_rate': 6.630999213217939e-06, 'epoch': 1.34}
{'loss': 1.8317, 'learning_rate': 5.844217151848939e-06, 'epoch': 1.42}
{'loss': 1.8275, 'learning_rate': 5.057435090479937e-06, 'epoch': 1.49}
{'loss': 1.8305, 'learning_rate': 4.270653029110937e-06, 'epoch': 1.57}
{'loss': 1.8277, 'learning_rate': 3.485444531864674e-06, 'epoch': 1.65}
{'loss': 1.8167, 'learning_rate': 2.6986624704956728e-06, 'epoch': 1.73}
{'loss': 1.8113, 'learning_rate': 1.911880409126672e-06, 'epoch': 1.81}
{'loss': 1.8056, 'lea

/home/odhi/anaconda3/envs/sentiment-analysis-transformers-pytorch/lib/python3.11/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1589 [00:00<?, ?it/s]

{'eval_loss': 1.600104570388794, 'eval_bleu': 5.6947, 'eval_gen_len': 17.5664, 'eval_runtime': 286.076, 'eval_samples_per_second': 88.847, 'eval_steps_per_second': 5.554, 'epoch': 2.0}
{'train_runtime': 2511.7092, 'train_samples_per_second': 80.955, 'train_steps_per_second': 5.06, 'train_loss': 1.8759875762003035, 'epoch': 2.0}


TrainOutput(global_step=12710, training_loss=1.8759875762003035, metrics={'train_runtime': 2511.7092, 'train_samples_per_second': 80.955, 'train_steps_per_second': 5.06, 'train_loss': 1.8759875762003035, 'epoch': 2.0})

In [12]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/taufiqodhi/my_awesome_opus_books_model/tree/main/'

In [10]:
from transformers import pipeline


text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."
translator = pipeline("translation", model="taufiqodhi/my_awesome_opus_books_model")
translator(text)

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

/home/odhi/anaconda3/envs/sentiment-analysis-transformers-pytorch/lib/python3.11/site-packages/transformers/pipelines/__init__.py:1049: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'Legumes partagent des ressources avec des bactéries fixatrices de nitrogen.'}]

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("taufiqodhi/my_awesome_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids


In [12]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("my_awesome_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)